# The Super League
If you are a fan of football or have been following any of the latest news then you would have probably heard about the proposed new 'Super League' that was planned with a selection of European clubs. This would have involved a group of 12 teams playing in a competition each year without having to qualify or be relegated. The lack of  fair competition between other clubs has caused an uproar among fans, players, media outlets, football associations, and even the UK government! 

The 'big 6' English teams to propose the Super League were Arsenal, Chelsea, Liverpool, Manchester United, Manchester City, and Tottenham Hotspur.

One of the ideas to try and discourage the clubs from proceeding with the new league, was to threaten the English teams with being expelled from the English Premier League. The challenge this week is to try and understand how the current league table would change if these clubs were to be 'kicked out'.

1. Calculate the Total Points for each team. The points are as follows: 
    * Win - 3 Points
    * Draw - 1 Point
    * Lose - 0 Points
2. Calculate the goal difference for each team. Goal difference is the difference between goals scored and goals conceded. 
3. Calculate the current rank/position of each team. This is based on Total Points (high to low) and in a case of a tie then Goal Difference (high to low).
4. Assuming that the 'Big 6' didn't play any games this season, recalculate the league table.
5. After removing the 6 clubs, how has the position changed for the remaining clubs?
6. The updated league table is the second output

In [74]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from pandasgui import show

In [25]:
df_games = pd.read_csv('PL Fixtures.csv')
df_games.dropna(inplace=True)
df_games[['Home Goals', 'Away Goals']] = df_games['Result'].str.split('-', expand=True).astype('int')

df_games['Winner'] = np.where(df_games['Home Goals'] > df_games['Away Goals'], df_games['Home Team'], np.where(df_games['Home Goals'] == df_games['Away Goals'], 'tie', df_games['Away Team']))

In [26]:
df_games['Team'] = df_games[['Home Team', 'Away Team']].values.tolist()
df_games_per_team = df_games.explode('Team')
df_games_per_team

,Round Number,Date,Location,Home Team,Away Team,Result,Home Goals,Away Goals,Winner,Team
0,1,12/09/2020 12:30,Craven Cottage,Fulham,Arsenal,0 - 3,0,3,Arsenal,Fulham
0,1,12/09/2020 12:30,Craven Cottage,Fulham,Arsenal,0 - 3,0,3,Arsenal,Arsenal
1,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0,Crystal Palace,Crystal Palace
1,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,1 - 0,1,0,Crystal Palace,Southampton
2,1,12/09/2020 17:30,Anfield,Liverpool,Leeds,4 - 3,4,3,Liverpool,Liverpool
...,...,...,...,...,...,...,...,...,...,...
315,32,18/04/2021 13:30,Emirates Stadium,Arsenal,Fulham,1 - 1,1,1,tie,Fulham
316,32,18/04/2021 16:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1,Man Utd,Man Utd
316,32,18/04/2021 16:00,Old Trafford,Man Utd,Burnley,3 - 1,3,1,Man Utd,Burnley
317,32,19/04/2021 20:00,Elland Road,Leeds,Liverpool,1 - 1,1,1,tie,Leeds


In [27]:
df_games_per_team['Result'] = np.where(df_games_per_team['Winner'] == df_games_per_team['Team'], 3, np.where(df_games_per_team['Winner'] == 'tie', 1, 0))
df_games_per_team['Goal Difference'] = np.where(df_games_per_team['Team'] == df_games_per_team['Home Team'], df_games_per_team['Home Goals'] - df_games_per_team['Away Goals'], df_games_per_team['Away Goals'] - df_games_per_team['Home Goals'])

In [43]:
df_standings = df_games_per_team.groupby('Team', as_index=False).agg(Games=('Team','count'), Points=('Result','sum'), Difference=('Goal Difference', 'sum'))
df_standings['Rank'] = df_standings.sort_values('Difference', ascending=False)['Points'].rank(ascending=False, method='first')
df_standings.sort_values(by='Rank')

,Team,Games,Points,Difference,Rank
11,Man City,32,74,44,1.0
12,Man Utd,32,66,29,2.0
9,Leicester,31,56,18,3.0
18,West Ham,32,55,11,4.0
4,Chelsea,31,54,19,5.0
10,Liverpool,32,53,16,6.0
16,Spurs,32,50,17,7.0
6,Everton,31,49,3,8.0
0,Arsenal,32,46,8,9.0
8,Leeds,32,46,0,10.0


In [76]:
big_six_teams = ['Arsenal', 'Chelsea', 'Liverpool', 'Man Utd', 'Man City', 'Spurs']
df_games_per_team_WO_big6 = df_games_per_team.loc[(~df_games_per_team['Home Team'].isin(big_six_teams))&(~df_games_per_team['Away Team'].isin(big_six_teams))].copy()
df_games_per_team_WO_big6

,Round Number,Date,Location,Home Team,Away Team,Result,Home Goals,Away Goals,Winner,Team,Goal Difference
1,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,3,1,0,Crystal Palace,Crystal Palace,1
1,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,0,1,0,Crystal Palace,Southampton,-1
3,1,12/09/2020 20:00,London Stadium,West Ham,Newcastle,0,0,2,Newcastle,West Ham,-2
3,1,12/09/2020 20:00,London Stadium,West Ham,Newcastle,3,0,2,Newcastle,Newcastle,2
4,1,13/09/2020 14:00,The Hawthorns,West Brom,Leicester,0,0,3,Leicester,West Brom,-3
...,...,...,...,...,...,...,...,...,...,...,...
310,31,12/04/2021 20:15,Amex Stadium,Brighton,Everton,1,0,0,tie,Everton,0
312,32,17/04/2021 12:30,St. James' Park,Newcastle,West Ham,3,3,2,Newcastle,Newcastle,1
312,32,17/04/2021 12:30,St. James' Park,Newcastle,West Ham,0,3,2,Newcastle,West Ham,-1
314,32,17/04/2021 20:15,Molineux Stadium,Wolves,Sheffield Utd,3,1,0,Wolves,Wolves,1


In [77]:
df_standings_WO_big6 = df_games_per_team_WO_big6.groupby('Team', as_index=False).agg(Games=('Team','count'), Points=('Result','sum'), Difference=('Goal Difference', 'sum'))
df_standings_WO_big6['Rank'] = df_standings_WO_big6.sort_values('Difference', ascending=False)['Points'].rank(ascending=False, method='first')
df_standings_WO_big6.sort_values(by='Rank')

,Team,Games,Points,Difference,Rank
12,West Ham,21,49,21,1.0
7,Leicester,22,40,15,2.0
6,Leeds,22,39,11,3.0
0,Aston Villa,22,34,7,4.0
4,Everton,21,34,4,5.0
3,Crystal Palace,22,32,0,6.0
8,Newcastle,23,32,-4,7.0
10,Southampton,21,30,1,8.0
13,Wolves,22,30,-1,9.0
1,Brighton,22,26,0,10.0
